<img width="600" height="200" src="../../datasets/redes.png">

# Animations with Earth Engine
## Purpose
The purpose of this notebook is to create animations from Earth Engine datasets.

## Setup
### Library import

In [1]:
import os
from ipywidgets import interact
import matplotlib.pyplot as plt

from vizzDL import animations
from vizzDL.models.CNN.super_resolution import srgan

## **Create animation**
**Create `animations` object**

In [3]:
animation = animations.Animation()

**Select region**

In [6]:
geometry = {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.206748, 46.034209],
      [-122.206748, 46.206846],
      [-122.010182, 46.206846],
      [-122.010182, 46.034209],
      [-122.206748, 46.034209]]]}}]}

animation.select_region(instrument='Landsat', geometry=geometry, lat=46.116, lon=-122.1149, zoom=11)      

In [4]:
geometry =  {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[48.831181, -13.983606],
      [48.831181, -13.856368],
      [48.97221, -13.856368],
      [48.97221, -13.983606],
      [48.831181, -13.983606]]]}}]}

animation.select_region(instrument='Landsat', geometry=geometry, lat=-13.95, lon=48.95, zoom=10)

Map(center=[-13.95, 48.95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

### Time-lapse

**Save images as `PNG`s**

In [8]:
#animation.save_frames_as_PGNs(folder_path = '../../datasets/processed/GeoTIFFs/', region_name = 'Saint_Helens', start_year=1995, stop_year=2019)
animation.save_frames_as_PGNs(folder_path = '../../datasets/processed/GeoTIFFs/', region_name = 'Tsaratanana', start_year=2016, stop_year=2021)

Image number: 0
Image number: 1
Image number: 2
Image number: 3
Image number: 4


**Create animation as a Numpy array**

In [5]:
#video = animation.video_as_array(start_year=1995, stop_year=2019)
video = animation.video_as_array(start_year=2001, stop_year=2021)

Image number: 0
Image number: 1
Image number: 2
Image number: 3
Image number: 4
Image number: 5
Image number: 6
Image number: 7
Image number: 8
Image number: 9
Image number: 10
Image number: 11
Image number: 12
Image number: 13
Image number: 14
Image number: 15
Image number: 16
Image number: 17
Image number: 18
Image number: 19
Image number: 20


In [6]:
@interact(frame=(0, video.shape[0]-1))
def show_frame(frame=0):
    fig, ax = plt.subplots(figsize=(10,10))

    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=20), Output()), _dom_classes=('widget-intera…

### Prediction
**Create model**

In [37]:
# Location of model weights
weights_dir = '../../datasets/processed/Models/L8_S2_SR_x3/srgan_generator_L8_to_S2_x3'
weights_file = lambda filename: os.path.join(weights_dir, filename)

pre_generator = srgan.Generator(input_shape=(None, None, 3), scale=3).generator()
pre_generator.load_weights(weights_file('model_weights.h5'))

**predict**

In [38]:
prediction = animation.predict(model = pre_generator, norm_range=[[0,1], [-1,1]])

1/1 [==============================] - 0s 304ms/step


In [39]:
@interact(frame=(0, video.shape[0]-1))
def show_frame(frame=0):
    fig, ax = plt.subplots(1, 2, figsize=(20,20))

    ax[0].imshow(video[frame,:,:,:])
    ax[0].set_title('LR')

    ax[1].imshow(prediction[frame,:,:,:])
    ax[1].set_title('SR (PRE)')

interactive(children=(IntSlider(value=0, description='frame', max=20), Output()), _dom_classes=('widget-intera…

### Create tiles
**Create tiles**

In [45]:
animation.create_animated_tiles(folder_path = '../../datasets/processed/Tiles/', region_name = 'Tsaratanana', minZ = 10, maxZ=14, save_GeoTIFF=True)

Sentinel
Create tiles per frame:
  year #2016
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
  year #2017
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
  year #2018
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
  year #2019
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
  year #2020
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
  year #2021
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10

IndexError: index 6 is out of bounds for axis 0 with size 6

### Create animation

In [7]:
animation.create_movie_from_array(folder_path = '../../datasets/processed/Animations/', region_name = 'Tsaratanana', output_format='mp4')